# 🏦 Mini Projet : Système de Gestion de Compte Bancaire

**Étape 1 : Version naïve (sans design pattern)**

*Code*

In [22]:
class Compte:
    def __init__(self):
        self.solde = 0
        self.historique = []
        self.vue_solde = None
        self.vue_operations = None
        self.vue_alerte = None

    def set_vues(self, vue_solde, vue_operations, vue_alerte):
        self.vue_solde = vue_solde
        self.vue_operations = vue_operations
        self.vue_alerte = vue_alerte

    def deposer(self, montant):
        self.solde += montant
        self.historique.append(f"Dépôt: +{montant}")
        if self.vue_solde:
            self.vue_solde.afficher_solde(self.solde)
        if self.vue_operations:
            self.vue_operations.afficher_operation(f"+{montant}")
        if self.vue_alerte:
            self.vue_alerte.verifier(self.solde)

    def retirer(self, montant):
        self.solde -= montant
        self.historique.append(f"Retrait: -{montant}")
        if self.vue_solde:
            self.vue_solde.afficher_solde(self.solde)
        if self.vue_operations:
            self.vue_operations.afficher_operation(f"-{montant}")
        if self.vue_alerte:
            self.vue_alerte.verifier(self.solde)


class VueSolde:
    def afficher_solde(self, solde):
        print(f"[SOLDE] Solde actuel: {solde}dt")


class VueOperations:
    def afficher_operation(self, operation):
        print(f"[OPERATION] {operation}dt")


class VueAlerte:
    def verifier(self, solde):
        if solde < 0:
            print("[ALERTE] Solde négatif!")
        if solde > 1000:
            print("[ALERTE] Solde élevé, pensez à investir!")


# Test de la version naïve
print("=== ÉTAPE 1 : Version naïve ===")
compte1 = Compte()
vs = VueSolde()
vo = VueOperations()
va = VueAlerte()

compte1.set_vues(vs, vo, va)

compte1.deposer(500)
compte1.retirer(200)
compte1.retirer(400)

print(f"Historique du compte: {compte1.historique}")
print()

=== ÉTAPE 1 : Version naïve ===
[SOLDE] Solde actuel: 500dt
[OPERATION] +500dt
[SOLDE] Solde actuel: 300dt
[OPERATION] -200dt
[SOLDE] Solde actuel: -100dt
[OPERATION] -400dt
[ALERTE] Solde négatif!
Historique du compte: ['Dépôt: +500', 'Retrait: -200', 'Retrait: -400']



**Étape 2 : Singleton**

*Code*

In [21]:
class CompteSingleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            print("Création de la seule instance du compte (Singleton)")
            cls._instance = super().__new__(cls)
            cls._instance.solde = 0
            cls._instance.historique = []
        return cls._instance

    def deposer(self, montant):
        self.solde += montant
        self.historique.append(f"Dépôt: +{montant}")
        print(f"Dépôt de {montant}dt effectué")

    def retirer(self, montant):
        if self.solde >= montant:
            self.solde -= montant
            self.historique.append(f"Retrait: -{montant}")
            print(f"Retrait de {montant}dt effectué")
        else:
            print("Fonds insuffisants")

    def afficher_solde(self):
        print(f"Solde actuel: {self.solde}dt")

    def afficher_historique(self):
        print("Historique des opérations:")
        for op in self.historique:
            print(f"  - {op}")


# Test du Singleton
print("=== ÉTAPE 2 : Singleton ===")
compte_a = CompteSingleton()
compte_a.deposer(1000)
compte_a.afficher_solde()

compte_b = CompteSingleton()  # Même instance!
compte_b.retirer(300)
compte_b.afficher_solde()

print("Vérification si c'est la même instance:")
print(f"compte_a est compte_b ? {compte_a is compte_b}")
print(f"ID de compte_a: {id(compte_a)}")
print(f"ID de compte_b: {id(compte_b)}")

compte_a.afficher_historique()
print()

=== ÉTAPE 2 : Singleton ===
Création de la seule instance du compte (Singleton)
Dépôt de 1000dt effectué
Solde actuel: 1000dt
Retrait de 300dt effectué
Solde actuel: 700dt
Vérification si c'est la même instance:
compte_a est compte_b ? True
ID de compte_a: 137567319786224
ID de compte_b: 137567319786224
Historique des opérations:
  - Dépôt: +1000
  - Retrait: -300



**Étape 3 : Observer**

*Sujet Observable(Code)*




In [7]:
class Observable:
    def __init__(self):
        self.observers = []

    def ajouter_observer(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)

    def retirer_observer(self, observer):
        if observer in self.observers:
            self.observers.remove(observer)

    def notifier(self, type_op, montant, solde):
        for observer in self.observers:
            observer.mettre_a_jour(type_op, montant, solde)


*Compte Observable(Code)*

In [8]:
class CompteObservable(Observable):
    def __init__(self):
        super().__init__()
        self.solde = 0
        self.historique = []

    def deposer(self, montant):
        self.solde += montant
        self.historique.append(('depot', montant))
        self.notifier('depot', montant, self.solde)

    def retirer(self, montant):
        if self.solde >= montant:
            self.solde -= montant
            self.historique.append(('retrait', montant))
            self.notifier('retrait', montant, self.solde)
        else:
            print("Fonds insuffisants")
            self.notifier('retrait_echoue', montant, self.solde)


*Observateurs(Code)*

In [9]:
class ObserverSolde:
    def mettre_a_jour(self, type_op, montant, solde):
        print(f"[OBS-SOLDE] Nouveau solde: {solde}dt (après {type_op} de {montant}dt)")


class ObserverOperations:
    def __init__(self):
        self.operations = []

    def mettre_a_jour(self, type_op, montant, solde):
        operation = f"{type_op}: {montant}dt"
        self.operations.append(operation)
        print(f"[OBS-OPERATIONS] {operation}")


class ObserverAlerte:
    def mettre_a_jour(self, type_op, montant, solde):
        if solde < 0:
            print("[OBS-ALERTE] ALERTE: Solde négatif!")
        elif solde < 50:
            print("[OBS-ALERTE] Attention: solde faible")
        elif solde > 5000:
            print("[OBS-ALERTE] Solde élevé détecté")


*Exemple*


In [10]:
if __name__ == "__main__":
    compte = CompteObservable()

    obs_solde = ObserverSolde()
    obs_ops = ObserverOperations()
    obs_alerte = ObserverAlerte()

    compte.ajouter_observer(obs_solde)
    compte.ajouter_observer(obs_ops)
    compte.ajouter_observer(obs_alerte)

    # Simulation d'opérations
    compte.deposer(800)
    compte.retirer(150)
    compte.retirer(700)  # Fonds insuffisants

    print("\nToutes les opérations enregistrées:", obs_ops.operations)


[OBS-SOLDE] Nouveau solde: 800dt (après depot de 800dt)
[OBS-OPERATIONS] depot: 800dt
[OBS-SOLDE] Nouveau solde: 650dt (après retrait de 150dt)
[OBS-OPERATIONS] retrait: 150dt
Fonds insuffisants
[OBS-SOLDE] Nouveau solde: 650dt (après retrait_echoue de 700dt)
[OBS-OPERATIONS] retrait_echoue: 700dt

Toutes les opérations enregistrées: ['depot: 800dt', 'retrait: 150dt', 'retrait_echoue: 700dt']


**Étape 4 : MVC**

*Modèle(Code)*

In [15]:
class CompteMVC:
    def __init__(self):
        self.solde = 0
        self.historique = []

    def deposer(self, montant):
        self.solde += montant
        self.historique.append(('depot', montant))
        return self.solde

    def retirer(self, montant):
        if self.solde >= montant:
            self.solde -= montant
            self.historique.append(('retrait', montant))
            return self.solde
        else:
            raise ValueError("Fonds insuffisants")


*Vues(Code)*

In [16]:
class VueConsole:
    def afficher_menu(self):
        print("\n=== MENU BANCAIRE ===")
        print("1. Déposer")
        print("2. Retirer")
        print("3. Afficher solde")
        print("4. Afficher historique")
        print("5. Quitter")

    def afficher_solde(self, solde):
        print(f"Solde: {solde}dt")

    def afficher_historique(self, historique):
        print("Historique:")
        for type_op, montant in historique:
            print(f"  {type_op}: {montant}dt")

    def demander_montant(self, operation):
        try:
            montant = float(input(f"Montant à {operation}: "))
            return montant
        except ValueError:
            print("Montant invalide")
            return None

    def afficher_message(self, message):
        print(message)


*Contrôleur(Code)*

In [17]:
class Controleur:
    def __init__(self):
        self.modele = CompteMVC()
        self.vue = VueConsole()

    def demarrer(self):
        while True:
            self.vue.afficher_menu()
            choix = input("Choix: ")

            if choix == "1":
                montant = self.vue.demander_montant("déposer")
                if montant and montant > 0:
                    nouveau_solde = self.modele.deposer(montant)
                    self.vue.afficher_message(f"Dépôt effectué. Nouveau solde: {nouveau_solde}dt")
                else:
                    self.vue.afficher_message("Montant invalide")

            elif choix == "2":
                montant = self.vue.demander_montant("retirer")
                if montant and montant > 0:
                    try:
                        nouveau_solde = self.modele.retirer(montant)
                        self.vue.afficher_message(f"Retrait effectué. Nouveau solde: {nouveau_solde}dt")
                    except ValueError as e:
                        self.vue.afficher_message(str(e))
                else:
                    self.vue.afficher_message("Montant invalide")

            elif choix == "3":
                self.vue.afficher_solde(self.modele.solde)

            elif choix == "4":
                self.vue.afficher_historique(self.modele.historique)

            elif choix == "5":
                print("Au revoir!")
                break

            else:
                print("Choix invalide")


*Exemple d’utilisation*

In [18]:
if __name__ == "__main__":
    # Simulation simple sans input()
    modele_test = CompteMVC()
    vue_test = VueConsole()

    print("\nSimulation d'opérations:")
    modele_test.deposer(500)
    vue_test.afficher_solde(modele_test.solde)

    modele_test.retirer(200)
    vue_test.afficher_solde(modele_test.solde)

    vue_test.afficher_historique(modele_test.historique)

    # Pour tester l'interface complète avec input():
    # controleur = Controleur()
    # controleur.demarrer()



Simulation d'opérations:
Solde: 500dt
Solde: 300dt
Historique:
  depot: 500dt
  retrait: 200dt
